USING VERTEX AI MODEL FOR EMBEDDING

In [68]:
from typing import Any, List, Dict
from sklearn.cluster import KMeans
from db_connections.pg_sql import session, Ticket
import numpy as np
import pandas as pd
import vertexai
from vertexai.language_models import TextEmbeddingModel
import os


In [69]:
PROJECT_ID = "decent-era-442806-n3"
LOCATION = "us-central1"  # Your Vertex AI region
EMBEDDING_DIM = 768
N_CLUSTERS = 2
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:/codes/langGraph_venv/embedding_POC/credentials.json"
os.environ["INCIDENTS_CSV_PATH"] = "D:/codes/langGraph_venv/embedding_POC/mock_incidents_100.csv"

In [70]:
vertexai.init(project=PROJECT_ID, location="us-central1")


In [92]:
sample_tickets = [
    {"ticket_id": "1", "text": "Cannot connect to the company VPN, it keeps saying authentication failed.", "category": "Network"},
    {"ticket_id": "2", "text": "My Microsoft Outlook is not opening, it crashes on startup.", "category": "Software"},
    {"ticket_id": "3", "text": "The new laptop I received won't boot up, it shows a black screen.", "category": "Hardware"},
    {"ticket_id": "4", "text": "I'm unable to access the shared drive, it says permission denied.", "category": "Network"},
    {"ticket_id": "5", "text": "Excel is freezing frequently when I work with large spreadsheets.", "category": "Software"},
    {"ticket_id": "6", "text": "The printer on the 3rd floor is not working, documents are stuck in queue.", "category": "Hardware"},
    {"ticket_id": "7", "text": "How do I reset my domain password?", "category": "Account"},
]

In [71]:
print("Starting data processing with psycopg2...")

# 1. Fetch Mock Incidents

INCIDENTS_CSV_PATH = os.getenv("INCIDENTS_CSV_PATH")
print(f"Loading incidents from CSV: {INCIDENTS_CSV_PATH}...")

try:
    # pd.read_csv automatically infers types, but we explicitly parse dates
    incidents_df = pd.read_csv(
        INCIDENTS_CSV_PATH,
        parse_dates=['created_at', 'updated_at'], # Parse these columns as datetime objects
        dayfirst=True # Adjust based on your date format (e.g., True for DD/MM/YYYY)
    )
    print(f"Loaded {len(incidents_df)} incidents from CSV.")
    print("First 5 rows of loaded data:")
    print(incidents_df.head())
except FileNotFoundError:
    print(f"Error: CSV file not found at {INCIDENTS_CSV_PATH}. Please check your .env file path.")
    exit(1)
except Exception as e:
    print(f"Error loading CSV: {e}")
    exit(1)


Starting data processing with psycopg2...
Loading incidents from CSV: D:/codes/langGraph_venv/embedding_POC/mock_incidents_100.csv...
Loaded 100 incidents from CSV.
First 5 rows of loaded data:
  ticket_id                                               text  \
0  INC-1000  Troubleshooting Network connectivity issue. Ro...   
1  INC-1001  Investigating Network connectivity issue. Affe...   
2  INC-1002  Urgent: API endpoint timeout impacts ERP avail...   
3  INC-1003  Investigating Server CPU spike. Affecting HR u...   
4  INC-1004  Urgent: Printer not responding impacts ERP ava...   

                   created_at                  updated_at    status priority  
0  2024-06-12 06:56:43.002475  2024-06-12 14:56:43.002475  Resolved   Medium  
1  2025-02-13 22:56:43.002475  2025-02-15 02:56:43.002475      Open   Medium  
2  2024-11-26 12:56:43.002475  2024-11-27 18:56:43.002475    Closed      Low  
3  2024-08-18 15:56:43.002475  2024-08-19 07:56:43.002475      Open   Medium  
4  2025-03-17 

In [72]:
def get_vertex_embedding(texts: List[str]) -> List[List[float]]:
    """Generate embeddings 
    using Vertex AI Gecko model."""
    embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
    embeddings = embedding_model.get_embeddings(texts)
    # embeddings is a list of Embedding objects; extract embedding vector
    return [emb.values for emb in embeddings]

INC-1000


In [106]:

def process_and_store_to_pgvector(kmeans: KMeans, tickets: List[Dict[str, Any]] | None =None, tickets_df:pd.DataFrame | None = None ):
    if tickets: 
        texts = [ticket['text'] for ticket in tickets]
    if tickets_df is not None:
        texts = tickets_df['text'].tolist()
    print("Generating")
    embeddings = get_vertex_embedding(texts)
    print(f"generated embedding for {len(embeddings)} ticketsa")
    cluster_ids = kmeans.fit_predict(embeddings)
    print("Kmeans cluster ids", cluster_ids)
    session.rollback()
    if tickets:
        for i, ticket in enumerate(tickets):
            embedding = np.array(embeddings[i], dtype=np.float32)
            new_ticket = Ticket(
                ticket_id=ticket['ticket_id'],
                text=ticket['text'],
                embedding=embedding,
                cluster_id=int(cluster_ids[i])
            )
            session.add(new_ticket)
        print(f"Inserted {len(tickets)} tickets into PostgreSQL with pgvector.")
    if tickets_df is not None:
        for i, row in tickets_df.iterrows():
            embedding = np.array(embeddings[i], dtype=np.float32)
            new_ticket = Ticket(
                ticket_id=row['ticket_id'],
                text=row['text'],
                embedding=embedding,
                cluster_id=int(cluster_ids[i])
            )
            session.add(new_ticket)
        print(f"Inserted {len(incidents_df)} tickets into PostgreSQL with pgvector.")


    session.commit()
   

In [107]:
# from sqlalchemy import text
# session.rollback() 
# # sql = text("""
# #        SELECT id, ticket_id, text,
# #                embedding <-> (:query_embedding)::vector AS distance
# #         FROM tickets
# #         ORDER BY embedding <-> (:query_embedding)::vector
# #         LIMIT :top_k;
# #     """)
# sql_query_cosine = text("""
#     SELECT id, ticket_id, text,
#            embedding <=> (:query_embedding)::vector as distance
#     FROM tickets
#     ORDER BY distance ASC 
#     LIMIT :top_k;
#     """)
# top_k=3
# result = session.execute(sql_query_cosine,{"query_embedding": query_embedding.tolist(), "top_k": top_k})
# session.rollback()
# result2=session.execute(sql_query_cosine,{"query_embedding": query_, "top_k": top_k})


In [108]:
from sqlalchemy import text
def query_similar_from_pgvector(query: str, top_k: int = 3):
    query_embedding = np.array(get_vertex_embedding([query])[0], dtype=np.float32)

    session.rollback() 
    # sql = text("""
    #        SELECT id, ticket_id, text,
    #                embedding <-> (:query_embedding)::vector AS distance
    #         FROM tickets
    #         ORDER BY embedding <-> (:query_embedding)::vector
    #         LIMIT :top_k;
    #     """)
    sql_query_cosine = text("""
    SELECT id, ticket_id, text,
           embedding <=> (:query_embedding)::vector as distance
    FROM tickets
    ORDER BY distance ASC 
    LIMIT :top_k;
    """)
    print("Executinf seacrh query")
    result = session.execute(sql_query_cosine,{"query_embedding": query_embedding.tolist(), "top_k": top_k})

    print("Search Results:")
    for row in result:
        print(f"ID: {row.id}, Ticket ID: {row.ticket_id}, Text: {row.text}, Distance: {row.distance:.4f}")


In [115]:
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init='auto')
process_and_store_to_pgvector(tickets_df=incidents_df, kmeans=kmeans)
# process_and_store_to_pgvector(tickets=sample_tickets, kmeans=kmeans)


Generating
generated embedding for 100 ticketsa
Kmeans cluster ids [0 0 1 1 1 1 1 0 0 1 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1
 0 1 1 1 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 1
 1 0 0 1 0 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 1 1 0 0 1 1]
Inserted 100 tickets into PostgreSQL with pgvector.


In [114]:
print("\n--- Query Examples ---")
query_similar_from_pgvector("vpn authentication failed")



--- Query Examples ---
Executinf seacrh query
Search Results:
ID: 323, Ticket ID: 1, Text: Cannot connect to the company VPN, it keeps saying authentication failed., Distance: 0.1441
ID: 260, Ticket ID: INC-1037, Text: Resolved: User authentication failure after checking firewall rules., Distance: 0.3381
ID: 241, Ticket ID: INC-1018, Text: Troubleshooting User authentication failure. Root cause unknown., Distance: 0.3419


In [112]:
query_similar_from_pgvector("Storage capacity above 90%")

Executinf seacrh query
Search Results:
ID: 242, Ticket ID: INC-1019, Text: Frequent reports of Storage capacity alert for users in AMER., Distance: 0.4306
ID: 314, Ticket ID: INC-1091, Text: Frequent reports of Storage capacity alert for users in EMEA., Distance: 0.4380
ID: 277, Ticket ID: INC-1054, Text: Frequent reports of Storage capacity alert for users in APAC., Distance: 0.4436
